In [1]:
import random

import syft as sy
import torch as th
hook = sy.TorchHook(th)

from syft.frameworks.torch.he.fv.modulus import CoeffModulus
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.key_generator import KeyGenerator
from syft.frameworks.torch.he.fv.context import Context
from syft.frameworks.torch.he.fv.encryption_params import EncryptionParams
context = Context(EncryptionParams(64, CoeffModulus().create(64, [30, 30]), 64))

In [2]:
key_generator = KeyGenerator(context)
keys = key_generator.keygen()
relin_key = key_generator.get_relin_keys()

In [3]:
a = th.tensor([[1,2,3], [4,5,6]])
b = th.tensor([[4,5,6], [7,8,9]])
def encyptTensor(tensor):
    return tensor.int().encrypt(protocol="bfv", private_key=keys[0], context = context) 

def get_x_y():
    return encyptTensor(a), encyptTensor(b)

def decrypt_ct(ct):
    return ct.decrypt(private_key = keys[0])

def check_value(ab, ct):
    assert(th.equal(ab, ct))

In [4]:
x, y = get_x_y()

In [5]:
check_value(a, decrypt_ct(x))
check_value(b, decrypt_ct(y))

check_value(a + b, decrypt_ct(x + y))

check_value(a, decrypt_ct(x))
check_value(b, decrypt_ct(y))

In [6]:
check_value(a, decrypt_ct(x))
check_value(b, decrypt_ct(y))

assert (th.equal(a + b, decrypt_ct(x.add(y))))

check_value(a, decrypt_ct(x))
check_value(b, decrypt_ct(y))

In [7]:
check_value(a, decrypt_ct(x))
check_value(b, decrypt_ct(y))

check_value(a + b, decrypt_ct(th.add(x, y)))

check_value(a, decrypt_ct(x))
check_value(b, decrypt_ct(y))

In [8]:
check_value(a, decrypt_ct(x))
check_value(b, decrypt_ct(y))

check_value(a * b, decrypt_ct(x * y))

check_value(a, decrypt_ct(x))
check_value(b, decrypt_ct(y))

In [9]:
check_value(a, decrypt_ct(x))
check_value(b, decrypt_ct(y))

check_value(a * b, decrypt_ct(x.mul(y)))

check_value(a, decrypt_ct(x))
check_value(b, decrypt_ct(y))

In [10]:
check_value(a, decrypt_ct(x))
check_value(b, decrypt_ct(y))

check_value(a * b, decrypt_ct(th.mul(x, y)))

check_value(a, decrypt_ct(x))
check_value(b, decrypt_ct(y))

In [11]:
a = th.tensor([[1,2,3], [4,5,6]])
b = th.tensor([[4,5], [6,7], [7,8]])
x, y = get_x_y()

check_value(a, decrypt_ct(x))
check_value(b, decrypt_ct(y))

check_value(a.mm(b), decrypt_ct(x.mm(y, relin_key=relin_key)))

check_value(a, decrypt_ct(x))
check_value(b, decrypt_ct(y))

In [12]:
# check_value(a, decrypt_ct(x))
# check_value(b, decrypt_ct(y))

# check_value(th.mm(a, b), decrypt_ct(th.mm(x, y)))

# check_value(a, decrypt_ct(x))
# check_value(b, decrypt_ct(y))

In [15]:
a = th.tensor([1, 2, 3, 4]).int()
b = a.encrypt(protocol="bfv", private_key=keys[0], context = context, inplace = True)
print(b)
print(a)

BFVTensor>[<syft.frameworks.torch.he.fv.ciphertext.CipherText object at 0x1309e17f0>
tensor([1, 2, 3, 4], dtype=torch.int32)
